<a href="https://colab.research.google.com/github/Avipsa1/UPPP275-Notebooks/blob/main/Map_world_human_development_index_using_bokeh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install bokeh
!pip install geopandas

     |████████████████████████████████| 1.0 MB 5.0 MB/s 
     |████████████████████████████████| 16.7 MB 339 kB/s 
     |████████████████████████████████| 6.3 MB 70.5 MB/s 


Import all necessary libraries

In [7]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.models import ColumnDataSource,HoverTool,LogColorMapper
from bokeh.palettes import viridis
import geopandas as gpd

In [5]:
hd = pd.read_csv("human_development.csv", header=0)#, encoding="ISO-8859-1")

hd.head()

,HDI Rank,Country,Human Development Index (HDI),Life Expectancy at Birth,Expected Years of Education,Mean Years of Education,Gross National Income (GNI) per Capita,GNI per Capita Rank Minus HDI Rank
0,1.0,Norway,0.944,81.6,17.5,12.6,"64,992",5.0
1,2.0,Australia,0.935,82.4,20.2,13.0,"42,261",17.0
2,3.0,Switzerland,0.930,83.0,15.8,12.8,"56,431",6.0
3,4.0,Denmark,0.923,80.2,18.7,12.7,"44,025",11.0
4,5.0,Netherlands,0.922,81.6,17.9,11.9,"45,435",9.0


In [8]:
map_df = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
map_df.head()

,pop_est,continent,name,iso_a3,gdp_md_est,geometry
0,920938,Oceania,Fiji,FJI,8374.0,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,53950935,Africa,Tanzania,TZA,150600.0,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,603253,Africa,W. Sahara,ESH,906.5,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,35623680,North America,Canada,CAN,1674000.0,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,326625791,North America,United States of America,USA,18560000.0,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."


In [10]:
map_hdi = map_df.merge(hd, left_on = 'name', right_on = 'Country')
map_hdi.head()

,pop_est,continent,name,iso_a3,gdp_md_est,geometry,HDI Rank,Country,Human Development Index (HDI),Life Expectancy at Birth,Expected Years of Education,Mean Years of Education,Gross National Income (GNI) per Capita,GNI per Capita Rank Minus HDI Rank
0,920938,Oceania,Fiji,FJI,8374.0,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",90.0,Fiji,0.727,70.0,15.7,9.9,"7,493",21.0
1,35623680,North America,Canada,CAN,1674000.0,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",9.0,Canada,0.913,82.0,15.9,13.0,"42,155",11.0
2,18556698,Asia,Kazakhstan,KAZ,460700.0,"POLYGON ((87.35997 49.21498, 86.59878 48.54918...",56.0,Kazakhstan,0.788,69.4,15.0,11.4,"20,867",-1.0
3,29748859,Asia,Uzbekistan,UZB,202300.0,"POLYGON ((55.96819 41.30864, 55.92892 44.99586...",114.0,Uzbekistan,0.675,68.4,11.5,10.9,"5,567",10.0
4,6909701,Oceania,Papua New Guinea,PNG,28020.0,"MULTIPOLYGON (((141.00021 -2.60015, 142.73525 ...",158.0,Papua New Guinea,0.505,62.6,9.9,4.0,"2,463",-1.0


In [21]:
map_hdi = map_hdi.to_crs(3395)

In [11]:
def gpd_bokeh(df):
    """Convert geometries from geopandas to bokeh format"""
    nan = float('nan')
    lons = []
    lats = []
    for i,shape in enumerate(df.geometry.values):
        if shape.geom_type == 'MultiPolygon':
            gx = []
            gy = []
            ng = len(shape.geoms) - 1
            for j,member in enumerate(shape.geoms):
                xy = np.array(list(member.exterior.coords))
                xs = xy[:,0].tolist()
                ys = xy[:,1].tolist()
                gx.extend(xs)
                gy.extend(ys)
                if j < ng:
                    gx.append(nan)
                    gy.append(nan)
            lons.append(gx)
            lats.append(gy)

        else:     
            xy = np.array(list(shape.exterior.coords))
            xs = xy[:,0].tolist()
            ys = xy[:,1].tolist()
            lons.append(xs)
            lats.append(ys) 

    return lons,lats

In [22]:
lons, lats = gpd_bokeh(map_hdi)
source = ColumnDataSource(data=dict(
    	x=lons, 
    	y=lats,
        country = map_hdi['Country'],
        hdi = map_hdi['Human Development Index (HDI)']))

In [28]:
from bokeh.transform import linear_cmap
from bokeh.palettes import viridis
color_mapper = linear_cmap(field_name='hdi', palette=viridis(256) ,
                           low=min(map_hdi['Human Development Index (HDI)']) ,
                           high=max(map_hdi['Human Development Index (HDI)']))
TOOLS = "pan,wheel_zoom,reset,hover,save"

In [29]:
from bokeh.models import ColumnDataSource,ColorBar
map = figure(plot_width=800, plot_height=600,title="World Human Development Index Map", tools=TOOLS,)
map.patches('x', 'y', source=source, line_color="white", line_width=0.1, color=color_mapper)

map.select_one(HoverTool).tooltips = [
    ('HDI', '@hdi'),
    ('Country', '@country')
]

color_bar = ColorBar(color_mapper=color_mapper['transform'], width=16, location=(0,0))
map.add_layout(color_bar, 'right')


In [30]:
from bokeh.io import output_file, show,output_notebook
output_notebook()

In [31]:
output_file("world_hdi_map.html")
show(map)